In [1]:
import pymongo
from faker import Faker
from bson import ObjectId
from datetime import datetime, timedelta
import random
import os

mongo_uri = os.getenv("MONGO_URI")

# Initialize Faker
faker = Faker()

# Connect to MongoDB
try:
    client = pymongo.MongoClient(mongo_uri)

    # Select your database and collections
    db = client['healthcare_workshop']
    patients_collection = db['patients']
    appointments_collection = db['appointments']

    # Fetch all patient IDs
    patient_ids = patients_collection.distinct('patientID')  # Now using 'patientID' instead of '_id'

    # Function to validate appointment logic
    def validate_appointment(appointment):
        if appointment['start'] >= appointment['end']:
            print(f"Error: Appointment start time {appointment['start']} is after end time {appointment['end']}")
            return False
        return True

    # Function to check for overlapping appointments
    def check_for_overlaps(patient_id, new_appointment):
        overlap_query = {
            "patientID": patient_id,
            "$or": [
                {"start": {"$lt": new_appointment['end'], "$gte": new_appointment['start']}},
                {"end": {"$gt": new_appointment['start'], "$lte": new_appointment['end']}}
            ]
        }
        overlapping_count = appointments_collection.count_documents(overlap_query)
        return overlapping_count > 0

    # Generate Appointments with additional fields
    def generate_appointment(patient_id, previous_appointment_id=None):
        now = datetime.now()
        start_time = now + timedelta(days=random.randint(0, 60))
        end_time = start_time + timedelta(minutes=random.randint(15, 60))  # Duration between 15 mins and 1 hour
        
        doctor_info = {
            "doctorID": faker.uuid4(),
            "name": faker.name(),
            "specialty": random.choice(["Cardiology", "Neurology", "Orthopedics", "Pediatrics"]),
            "ratings": {
                "score": float(random.uniform(1.0, 5.0)),  # Convert Decimal to float
                "comments": faker.sentence()
            }
        }
        
        # Example of an array of services provided during the appointment
        services_provided = [
            {
                "serviceName": random.choice(["Blood Test", "X-Ray", "MRI", "Ultrasound"]),
                "serviceDate": start_time + timedelta(minutes=random.randint(0, 10))
            } for _ in range(random.randint(1, 3))
        ]
        
        return {
            "_id": ObjectId(),
            "patientID": patient_id,
            "status": random.choice(["booked", "completed", "canceled"]),
            "appointmentType": random.choice(["Consultation", "Follow-up", "Test"]),
            "start": start_time,
            "end": end_time,
            "previousAppointmentID": previous_appointment_id,
            "createdAt": now,
            "updatedAt": now,
            "doctor": doctor_info,  # Embedded document
            "services": services_provided,  # Array of embedded documents
            "appointmentNotes": faker.text(),  # Random appointment notes
            "patientSatisfaction": float(round(random.uniform(1, 5), 2)),  # Convert Decimal to float
            "diagnosis": random.choice(["Hypertension", "Diabetes", "Back Pain", "Flu", "Healthy"]),  # Random diagnosis
            "treatmentPlan": faker.paragraph(),  # Random treatment plan
            "location": {
                "type": "Point",
                "coordinates": [float(faker.longitude()), float(faker.latitude())]  # Convert Decimal to float
            }  # GeoJSON field for appointment location
        }

    # Insert multiple appointments
    def insert_appointments(num_appointments):
        appointments = []
        for _ in range(num_appointments):
            patient_id = random.choice(patient_ids)
            previous_appointment = random.choice(appointments) if appointments else None
            appointment = generate_appointment(
                patient_id,
                previous_appointment_id=previous_appointment["_id"] if previous_appointment else None
            )
            
            if validate_appointment(appointment) and not check_for_overlaps(appointment['patientID'], appointment):
                appointments.append(appointment)
            else:
                print(f"Skipped an invalid or overlapping appointment: {appointment}")
        
        if appointments:
            appointments_collection.insert_many(appointments)
            print(f"Successfully inserted {len(appointments)} appointments into MongoDB.")
        else:
            print("No valid appointments to insert.")

    if __name__ == "__main__":
        insert_appointments(1000)

except pymongo.errors.ServerSelectionTimeoutError as err:
    print("Connection error:", err)
except Exception as e:
    print("An unexpected error occurred:", e)

finally:
    client.close()  # Close the connection to MongoDB

Successfully inserted 1 appointments into MongoDB.
